In [116]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [117]:
res17 = pd.read_csv('electdata_2017.txt',sep=';')
res15 = pd.read_csv('electdata_2015.txt',sep=';')
res10 = pd.read_csv('electdata_2010ug.txt',sep=';')
res05 = pd.read_csv('electdata_2005nb.txt',sep=';')

res17['year'] = 2017
res15['year'] = 2015
res10['year'] = 2010
res05['year'] = 2005

res17['winner'] = 'CON'
res15['winner'] = 'CON'
res10['winner'] = 'CON'
res05['winner'] = 'CON'
for party in ['LAB','LIB','NAT','MIN','OTH','Green','UKIP']:
    res17.loc[(res17[party] >= res17.CON) & (res17[party] >= res17.LAB) & \
              (res17[party] >= res17.LIB) & (res17[party] >= res17.NAT) & \
              (res17[party] >= res17.Green) & (res17[party] >= res17.UKIP),'winner'] = party
    res15.loc[(res15[party] >= res15.CON) & (res15[party] >= res15.LAB) & \
              (res15[party] >= res15.LIB) & (res15[party] >= res15.NAT) & \
              (res15[party] >= res15.Green) & (res15[party] >= res15.UKIP),'winner'] = party
    res10.loc[(res10[party] >= res10.CON) & (res10[party] >= res10.LAB) & \
              (res10[party] >= res10.LIB) & (res10[party] >= res10.NAT) & \
              (res10[party] >= res10.Green) & (res10[party] >= res10.UKIP),'winner'] = party
    if party not in ['UKIP','Green']:
        res05.loc[(res05[party] >= res05.CON) & (res05[party] >= res05.LAB) & \
                  (res05[party] >= res05.LIB) & (res05[party] >= res05.NAT),'winner'] = party

res17['Turnout_pc'] = (res17['CON']+res17['LAB']+res17['LIB']+res17['UKIP']+
                       res17['NAT']+res17['Green']+res17['MIN']+res17['OTH']) / res17['Electorate']
res15['Turnout_pc'] = (res15['CON']+res15['LAB']+res15['LIB']+res15['UKIP']+
                       res15['NAT']+res15['Green']+res15['MIN']+res15['OTH']) / res15['Electorate']
res10['Turnout_pc'] = (res10['CON']+res10['LAB']+res10['LIB']+res10['UKIP']+
                       res10['NAT']+res10['Green']+res10['MIN']+res10['OTH']) / res10['Electorate']
res05['Turnout_pc'] = (res05['CON']+res05['LAB']+res05['LIB']+
                       res05['NAT']+res05['MIN']+res05['OTH']) / res05['Electorate']

for party in ['CON','LAB','LIB','UKIP','Green','NAT','MIN','OTH']:
    res17[party+'_pc'] = 100*res17[party] / (res17['Electorate']*res17['Turnout_pc'])
    res15[party+'_pc'] = 100*res15[party]/ (res15['Electorate']*res15['Turnout_pc'])
    res10[party+'_pc'] = 100*res10[party]/ (res10['Electorate']*res10['Turnout_pc'])
    if party not in ['UKIP','Green']:
        res05[party+'_pc'] = 100*res05[party] / (res05['Electorate']*res05['Turnout_pc'])

In [118]:
res17 = pd.merge(res17, res15, on=['Name','Region','County Area'], how='inner', suffixes=('','_prev'))
res15 = pd.merge(res15, res10, on=['Name','Region','County Area'], how='inner', suffixes=('','_prev'))
res10 = pd.merge(res10, res05, on=['Name','Region','County Area'], how='inner', suffixes=('','_prev'))
#Exclude NI regions because DUP are labelled as LIB which may be confusing
res17 = res17[res17.Region != 3]
res15 = res15[res15.Region != 3]
res10 = res10[res10.Region != 3]

In [119]:
training = pd.concat([res15[list(res10.columns)],res10])
test = res17

In [157]:
#Calculate some more things
def get_winner_pc_majority(row):
    allpcs = row[['CON_pc','LAB_pc','LIB_pc','Green_pc','UKIP_pc','NAT_pc','MIN_pc','OTH_pc']].values
    return row.winner_pc - np.sort(allpcs)[-2]

training['winner_pc'] = training.apply(lambda row: row[row.winner+'_pc'],axis=1)
test['winner_pc'] = test.apply(lambda row: row[row.winner+'_pc'],axis=1)
training['winner_majority_pc'] = training.apply(get_winner_pc_majority, axis=1)
test['winner_majority_pc'] = test.apply(get_winner_pc_majority, axis=1)

count    632.000000
mean      24.019887
std       16.448468
min        0.004782
25%        9.590799
50%       23.680349
75%       34.851301
max       77.140555
Name: winner_majority_pc, dtype: float64


In [120]:
baseline_preds = test.winner_prev
print 'Baseline 2017 accuracy: %.2f' % np.mean(baseline_preds == test.winner)

Baseline 2017 accuracy: 0.90


In [121]:
from sklearn.linear_model import LogisticRegression,SGDClassifier,Perceptron,LinearRegression
from sklearn.preprocessing import LabelEncoder
from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier
from sklearn import tree
from sklearn import ensemble
from sklearn.metrics import accuracy_score

In [122]:
le = LabelEncoder()
le.fit(training['winner_prev'].append(test['winner_prev']))
training['winner_prev_num'] = le.transform(training['winner_prev'])
test['winner_prev_num'] = le.transform(test['winner_prev'])

predictors = ['CON_pc_prev','LAB_pc_prev','NAT_pc_prev','LIB_pc_prev',
                    'winner_prev_num','Turnout_pc_prev']
X_train = training[predictors]
y_train = training['winner']
X_test = test[predictors]

for method, clf in zip(['log reg','sgd','perc','SVM','linSVM','KNN','DTree','RForest','AdaB10'],
               [LogisticRegression(), SGDClassifier(), Perceptron(), svm.SVC(), svm.LinearSVC(), 
                KNeighborsClassifier(n_neighbors=3), 
                tree.DecisionTreeClassifier(min_samples_split=10),
                ensemble.RandomForestClassifier(n_estimators=100, min_samples_split=10),
                ensemble.AdaBoostClassifier(n_estimators=10)]):
    clf.fit(X_train, y_train)
    preds = clf.predict(X_test)
    if method == 'RForest':
        rfclf = clf
        rfpreds1 = preds
        rftrainpreds1 = clf.predict(X_train)
    print '%10s: %.3f (%.0f%% are prev winner)' % (method,accuracy_score(preds, test.winner),
                                                100*accuracy_score(preds, test.winner_prev))

print '\nRF feature importances:'
for feature, importance in zip(X_train.columns,rfclf.feature_importances_):
    print '%15s: %.2f' % (feature, importance)
#There is no information beyond previous results, so best prediction is winner_prev
#Best are Perceptron or Random Forest at 87-88% (baseline is 90%)

   log reg: 0.854 (93% are prev winner)
       sgd: 0.796 (87% are prev winner)
      perc: 0.790 (85% are prev winner)
       SVM: 0.443 (41% are prev winner)
    linSVM: 0.801 (88% are prev winner)
       KNN: 0.870 (94% are prev winner)
     DTree: 0.839 (91% are prev winner)
   RForest: 0.862 (95% are prev winner)
    AdaB10: 0.829 (91% are prev winner)

RF feature importances:
    CON_pc_prev: 0.32
    LAB_pc_prev: 0.21
    NAT_pc_prev: 0.07
    LIB_pc_prev: 0.09
winner_prev_num: 0.21
Turnout_pc_prev: 0.10


In [123]:
#print rfpreds[rfpreds != res17.winner]
for party in ['CON','LAB','LIB','NAT','UKIP','Green']:
    print party, len([x for x in rfpreds1 if x == party]) - len([x for x in test.winner if x == party])
#I overpredict CON and NAT and underpredict LAB
#Polling will not help with this because the changes suggest more CON wins, and I have nothing on NAT

print '\n2010, 2015 training total predicted seats (error):'
for party in ['CON','LAB','LIB','NAT','UKIP','Green']:
    print '%s %i (%i)' % (party, len([x for x in rftrainpreds1 if x == party]), 
                          len([x for x in rftrainpreds1 if x == party]) - len([x for x in training.winner if x == party]))
print 'Accuracy = %.3f' % accuracy_score(rftrainpreds1, training.winner)
#In training I overestimate LAB

CON 34
LAB -50
LIB -10
NAT 27
UKIP 0
Green -1

2010, 2015 training total predicted seats (error):
CON 646 (9)
LAB 491 (5)
LIB 51 (-12)
NAT 68 (0)
UKIP 0 (-1)
Green 1 (-1)
Accuracy = 0.951


In [124]:
#Add opinion polling
polls = pd.read_csv('finalpolls.csv')
for party in ['CON','LAB','LIB']:
    res17['pollChange'+party] = float(polls.loc[polls.year==2017,party]) - float(polls.loc[polls.year==2015,party])
    res15['pollChange'+party] = float(polls.loc[polls.year==2015,party]) - float(polls.loc[polls.year==2010,party])
    res10['pollChange'+party] = float(polls.loc[polls.year==2010,party]) - float(polls.loc[polls.year==2005,party])

In [125]:
training = pd.concat([res15[list(res10.columns)],res10])
test = res17

le = LabelEncoder()
le.fit(training['winner_prev'].append(test['winner_prev']))
training['winner_prev_num'] = le.transform(training['winner_prev'])
test['winner_prev_num'] = le.transform(test['winner_prev'])

predictors = ['CON_pc_prev','LAB_pc_prev','NAT_pc_prev','LIB_pc_prev',
                    'winner_prev_num','Turnout_pc_prev',
             'pollChangeCON','pollChangeLAB','pollChangeLIB']
X_train = training[predictors]
y_train = training['winner']
X_test = test[predictors]

for method, clf in zip(['log reg','sgd','perc','SVM','linSVM','KNN','DTree','RForest','AdaB10'],
               [LogisticRegression(), SGDClassifier(), Perceptron(), svm.SVC(), svm.LinearSVC(), 
                KNeighborsClassifier(n_neighbors=3), 
                tree.DecisionTreeClassifier(min_samples_split=10),
                ensemble.RandomForestClassifier(n_estimators=100, min_samples_split=10),
                ensemble.AdaBoostClassifier(n_estimators=10)]):
    clf.fit(X_train, y_train)
    preds = clf.predict(X_test)
    if method == 'RForest':
        rfclf = clf
        rfpreds2 = preds
        rftrainpreds2 = clf.predict(X_train)
    print '%10s: %.3f (%.0f%% are prev winner)' % (method,accuracy_score(preds, test.winner),
                                                100*accuracy_score(preds, test.winner_prev))

print '\nRF feature importances:'
for feature, importance in zip(X_train.columns,rfclf.feature_importances_):
    print '%15s: %.2f' % (feature, importance)

   log reg: 0.532 (61% are prev winner)
       sgd: 0.802 (87% are prev winner)
      perc: 0.837 (91% are prev winner)
       SVM: 0.415 (37% are prev winner)
    linSVM: 0.491 (53% are prev winner)
       KNN: 0.854 (94% are prev winner)
     DTree: 0.835 (92% are prev winner)
   RForest: 0.873 (95% are prev winner)
    AdaB10: 0.829 (91% are prev winner)

RF feature importances:
    CON_pc_prev: 0.37
    LAB_pc_prev: 0.16
    NAT_pc_prev: 0.07
    LIB_pc_prev: 0.08
winner_prev_num: 0.19
Turnout_pc_prev: 0.06
  pollChangeCON: 0.03
  pollChangeLAB: 0.03
  pollChangeLIB: 0.02


In [126]:
#print rfpreds[rfpreds != res17.winner]
for party in ['CON','LAB','LIB','NAT','UKIP','Green']:
    print party, len([x for x in rfpreds2 if x == party]) - len([x for x in res17.winner if x == party])
    
print '\n2010, 2015 training total predicted seats (error):'
for party in ['CON','LAB','LIB','NAT','UKIP','Green']:
    print '%s %i (%i)' % (party, len([x for x in rftrainpreds2 if x == party]), 
                          len([x for x in rftrainpreds2 if x == party]) - len([x for x in training.winner if x == party]))
print 'Accuracy = %.3f' % accuracy_score(rftrainpreds2, training.winner)
#Poll changes in 2010 and 2015 show LAB decreasing so removes positive bias of LAB predictions

CON 37
LAB -46
LIB -11
NAT 21
UKIP 0
Green -1

2010, 2015 training total predicted seats (error):
CON 638 (1)
LAB 485 (-1)
LIB 59 (-4)
NAT 74 (6)
UKIP 0 (-1)
Green 1 (-1)
Accuracy = 0.959


In [127]:
print 'Changes in 2017 preds after including polls:'
for p1, p2 in zip(rfpreds1,rfpreds2):
    if p1 != p2:
        print p1,p2

Changes in 2017 preds after including polls:
LAB CON
LAB CON
LAB CON
LIB CON
CON LIB
NAT LAB
NAT LAB
NAT LAB
NAT LAB
NAT LAB
NAT LAB
NAT LAB
LIB NAT


In [128]:
#TODO: different approach, use poll changes to model changes in vote percentages, then
#  add these on to previous percentages to work out new winner. e.g. should probably
#  predict Green win in BP.
for party in ['CON','LAB','LIB']:
    res17['voteChange'+party] = res17[party+'_pc'] - res17[party+'_pc_prev']
    res15['voteChange'+party] = res15[party+'_pc'] - res15[party+'_pc_prev']
    res10['voteChange'+party] = res10[party+'_pc'] - res10[party+'_pc_prev']

In [129]:
from sklearn import metrics

training = pd.concat([res15[list(res10.columns)],res10])
test = res17

# le = LabelEncoder()
# le.fit(training['winner_prev'].append(test['winner_prev']))
# training['winner_prev_num'] = le.transform(training['winner_prev'])
# test['winner_prev_num'] = le.transform(test['winner_prev'])

predCON2017 = test['CON_pc_prev'] + test['pollChangeCON']
predLAB2017 = test['LAB_pc_prev'] + test['pollChangeLAB']
predLIB2017 = test['LIB_pc_prev'] + test['pollChangeLIB']

print 'Mean CON %% error baseline = %.2f, poll adjusted = %.2f' % (np.mean(test['CON_pc_prev']-test['CON_pc']), np.mean(predCON2017-test['CON_pc']))
print 'Mean LAB %% error baseline = %.2f, poll adjusted = %.2f' % (np.mean(test['LAB_pc_prev']-test['LAB_pc']), np.mean(predLAB2017-test['LAB_pc']))
print 'Mean LIB %% error baseline = %.2f, poll adjusted = %.2f' % (np.mean(test['LIB_pc_prev']-test['LIB_pc']), np.mean(predLIB2017-test['LIB_pc']))


Mean CON % error baseline = -5.89, poll adjusted = 4.91
Mean LAB % error baseline = -9.52, poll adjusted = -5.42
Mean LIB % error baseline = 0.60, poll adjusted = -2.50


In [170]:
#Some functions for scoring
def predPercentsAddPoll(row):
    outrow = row
    outrow['pred_CON_pc'] = min(max(outrow['CON_pc_prev'] + outrow['pollChangeCON'], 0), 100)
    outrow['pred_LAB_pc'] = min(max(outrow['LAB_pc_prev'] + outrow['pollChangeLAB'], 0), 100)
    outrow['pred_LIB_pc'] = min(max(outrow['LIB_pc_prev'] + outrow['pollChangeLIB'], 0), 100)
    #big3scale = sum(outrow[['pred_CON_pc','pred_LAB_pc','pred_LIB_pc']].values) / sum(outrow[['CON_pc_prev','LAB_pc_prev','LIB_pc_prev']].values)
    #print big3scale,sum(outrow[['pred_CON_pc','pred_LAB_pc','pred_LIB_pc']].values),sum(outrow[['CON_pc_prev','LAB_pc_prev','LIB_pc_prev']].values)
    outrow['pred_Green_pc'] = outrow['Green_pc_prev'] 
    outrow['pred_UKIP_pc'] = outrow['UKIP_pc_prev'] 
    outrow['pred_NAT_pc'] = outrow['NAT_pc_prev'] 
    outrow['pred_MIN_pc'] = outrow['MIN_pc_prev'] 
    outrow['pred_OTH_pc'] = outrow['OTH_pc_prev'] 
    #print sum(outrow[['CON_pc','LAB_pc','LIB_pc','Green_pc','UKIP_pc','NAT_pc','MIN_pc','OTH_pc']].values)
    divFact = sum(outrow[['pred_CON_pc','pred_LAB_pc','pred_LIB_pc','pred_Green_pc','pred_UKIP_pc','pred_NAT_pc','pred_MIN_pc','pred_OTH_pc']].values) / 100
    for party in ['CON','LAB','LIB','Green','UKIP','NAT','MIN','OTH']:
        outrow['pred_'+party+'_pc'] /= divFact
    #print sum(outrow[['pred_CON_pc','pred_LAB_pc','pred_LIB_pc','pred_Green_pc','pred_UKIP_pc','pred_NAT_pc','pred_MIN_pc','pred_OTH_pc']].values)
    return outrow

def getWinner(row, getPercentsMethod):
    outDF = getPercentsMethod(row)
    winIndex = np.argmax(outDF[['pred_CON_pc','pred_LAB_pc','pred_LIB_pc','pred_Green_pc','pred_UKIP_pc','pred_NAT_pc','pred_MIN_pc','pred_OTH_pc']].values)
    return ['CON','LAB','LIB','Green','UKIP','NAT','MIN','OTH'][winIndex]

def getWinnerBaseline(test):
    winIndex = np.argmax(test[['CON_pc_prev','LAB_pc_prev','LIB_pc_prev','Green_pc_prev','NAT_pc_prev','MIN_pc_prev','OTH_pc_prev']].values)
    return ['CON','LAB','LIB','Green','NAT','MIN','OTH'][winIndex]

#
def predictSeats(test):
    winners = test.apply(getWinner,axis=1,args=[predPercentsAddPoll])
    for party in ['CON','LAB','LIB','Green','NAT','MIN','OTH']:
        nSeats = len([p for p in winners if p == party])
        if nSeats > 0:
            print party,nSeats

def predictAndEvaluate(test, method):
    if method == 'baseline':
        predWinners = test.apply(getWinnerBaseline,axis=1)
    else:
        predWinners = test.apply(getWinner,axis=1,args=[method])
    trueWinners = test.winner
    nRight = 0
    for i in range(len(trueWinners)):
        if predWinners[i] == trueWinners[i]:
            nRight += 1
    return nRight

print 'Baseline gets %i/650 seats right' % predictAndEvaluate(test,'baseline')
print 'AddPoll gets %i/650 seats right' % predictAndEvaluate(test,predPercentsAddPoll)
#TODO: breakdown accuracy by party, winner_majority_pc_prev?

#TODO could use a model to better apply national poll changes to individual constituencies, e.g. grouped by winner_pc_majority or party vote pc

Baseline gets 568/650 seats right
AddPoll gets 551/650 seats right


In [164]:
#print test.head(12).apply(getWinner,axis=1,args=[predPercentsAddPoll])
print 'Baseline Lab % MAE: ',metrics.mean_absolute_error(test.LAB_pc_prev, test.LAB_pc)
print 'AddPoll Lab % MAE: ',metrics.mean_absolute_error(test.apply(predPercentsAddPoll,axis=1).pred_LAB_pc, test.LAB_pc)

Baseline Lab % MAE:  9.67627063736
AddPoll Lab % MAE:  9.59070117354
